## **Implementación**

In [45]:
KEY = 'tWFK491q'

In [46]:
def lcg(x_n, a, c, m):
    return (a*x_n + c) % m;

El LCG es un algoritmo clásico para generar secuencias de números pseudoaleatorios. Su funcionamiento se basa en la siguiente fórmula recursiva:

$$ x_{n+1} = (ax_n + c) \mod m $$

Donde:
- $ a $ es el multiplicador
- $ c $ es el incremento
- $ m $ es el módulo
- $ x_n $ es el valor actual
- $ x_{n+1} $ es el siguiente valor

Referencia: [Linear Congruential Generator (LCG)](https://en.wikipedia.org/wiki/Linear_congruential_generator)

In [47]:

def generate_keystream(key: str, message: str) -> bytes:
    # We need a value to start the sequence
    key_num = sum(ord(c) for c in key) 
    
    m = 2**31  
    a = 1103515245
    c = 12345
    
    value = (a*key_num + c) % m  
    keystream = []
    
    for _ in range(len(message)):
        keystream.append(value % 256)
        value = lcg(value, a, c, m)  


    return bytes(keystream)


Para cada byte del mensaje, se genera un nuevo valor pseudoaleatorio usando la función `lcg`, el resultado se toma módulo 256 (`value % 256`) para obtener un byte que se añade al `keystream`.

<div class="alert alert-block alert-info">
<b>Nota:</b> Se utilizan valores específicos para m, a y c que son los mismos que usa la implementación de `rand()` en glibc.
</div>

In [48]:
def encrypt_message(message: str, key: str) -> bytes:
    keystream = generate_keystream(key, message)
    return bytes([b ^ k for b, k in zip(message.encode(), keystream)])

def decrypt_message(encrypted_message: bytes, key: str) -> str:
    keystream = generate_keystream(key, encrypted_message)
    return bytes([b ^ k for b, k in zip(encrypted_message, keystream)]).decode()

In [49]:
decrypt_message(encrypt_message('This works!', KEY), KEY) == 'This works!'

True

## **Pruebas Unitarias**

In [55]:
import pandas as pd

# Different messages, same key.
test_messages = [
    "Hello, World!",
    "Cryptography is fascinating",
    "Special chars: !@#$%^&*()",
    "Numbers: 12345",
    "Empty string should work too: "
]

results = []
for msg in test_messages:
    encrypted = encrypt_message(msg, KEY)
    decrypted = decrypt_message(encrypted, KEY)
    status = "✅ Passed" if msg == decrypted else "❌ Failed"
    results.append({
        "Original Message": msg,
        "Encrypted (hex)": encrypted.hex()[:20] + "..." if len(encrypted) > 10 else encrypted.hex(),
        "Decrypted": decrypted,
        "Status": status
    })

df = pd.DataFrame(results)
display(HTML(df.to_html(escape=False, index=False, classes="table table-striped table-hover")))


# Same message, different keys.
message = "This is a test message"
keys = ["key1", "longkey123456", "short", "!@#$%^&*()"]
    
key_results = []
for key in keys:
    encrypted = encrypt_message(message, key)
    decrypted = decrypt_message(encrypted, key)
    status = "✅ Passed" if message == decrypted else "❌ Failed"
    key_results.append({
        "Key": key,
        "Encrypted (hex)": encrypted.hex()[:20] + "..." if len(encrypted) > 10 else encrypted.hex(),
        "Status": status
    })


key_df = pd.DataFrame(key_results)
display(HTML(key_df.to_html(escape=False, index=False, classes="table table-striped table-hover")))

Original Message,Encrypted (hex),Decrypted,Status
"Hello, World!",8004ea2b1bb132b48feb...,"Hello, World!",✅ Passed
Cryptography is fascinating,8b13ff3700f2759181e9...,Cryptography is fascinating,✅ Passed
Special chars: !@#$%^&*(),9b11e3241dfc7ec383f1...,Special chars: !@#$%^&*(),✅ Passed
Numbers: 12345,8614eb2511ef61d9c0a8...,Numbers: 12345,✅ Passed
Empty string should work too:,8d0cf6330dbd619792f0...,Empty string should work too:,✅ Passed


Key,Encrypted (hex),Status
key1,7fe04835275d2ef2c280...,✅ Passed
longkey123456,9b340c49cb219226a6d4...,✅ Passed
short,fd46a62f455398688026...,✅ Passed
!@#$%^&*(),414272cbb19f8444aca2...,✅ Passed


## **Preguntas**



*1. ¿Qué sucede cuando cambias la clave utilizada para generar el keystream?*


Cuando se cambia la clave, se obtiene un keystream completamente diferente.
Esto significa que el mismo mensaje se cifrará de manera totalmente distinta

In [51]:
mensaje = "Hola"
clave1 = "KEY1"
clave2 = "KEY2"

cifrado1 = encrypt_message(mensaje, clave1)
cifrado2 = encrypt_message(mensaje, clave2)

print(f"Key: {clave1} -> {cifrado1.hex().upper()}")
print(f"Key: {clave2} -> {cifrado2.hex().upper()}") 

Key: KEY1 -> 03472D87
Key: KEY2 -> F0FE9A96


*2. ¿Qué riesgos de seguridad existen si reutilizas el mismo keystream para cifrar dos mensajes diferentes?*

Es bastante peligroso usar el mismo keystream. Supongamos que tenemos dos mensajes M1 y M2:

\begin{align*}
    C_1 &= M_1 \oplus K \\
    C_2 &= M_2 \oplus K
\end{align*}

Podemos llegar a la siguiente equivalencia:

\begin{align*}
    C_1 \oplus C_2 &= (M_1 \oplus K) \oplus (M_2 \oplus K) \\
                     &= M_1 \oplus M_2
\end{align*}
Lo cual expone la información de los mensajes originales.

*3. ¿Cómo afecta la longitud del keystream a la seguridad del cifrado?*

Si el keystream es demasiado corto o se repite, se abren la posibilidad a ataques de reutilización de claves, como el conocido ataque de dos-time pad, en el que dos mensajes cifrados con el mismo keystream permiten derivar información del texto original. Además, un keystream predecible o cíclico puede facilitar análisis estadísticos y ataques de correlación, comprometiendo la integridad del cifrado.

Hay que utilizar generadores de keystream robustos que aseguren una secuencia larga, única e impredecible para cada sesión o mensaje. Esto no solo previene la repetición de patrones, sino que también dificulta a los atacantes la reconstrucción del keystream o la deducción de la clave secreta, garantizando una protección contra diversas técnicas criptográficas.


*4. ¿Qué consideraciones debes tener al generar un keystream en un entorno real?*

Lo primero sería usar un generador de números pseudoaleatorios criptográficamente seguro (CSPRNG) que garantice impredecibilidad y ausencia de patrones repetitivos. La semilla para dicho generador debe obtenerse de fuentes de alta entropía, como dispositivos de hardware o eventos del sistema, y manejarse de forma que no se exponga ni reutilice en múltiples sesiones.

En el código anterior, se usa un generador congruencial lineal (LCG) basado en la suma de los valores ASCII de la clave como semilla. Sin embargo, los LCG no son adecuados para criptografía, ya que producen secuencias predecibles y tienen períodos limitados, lo que los hace vulnerables a ataques de correlación y análisis estadístico.

Para mejorar la seguridad del keystream, se pueden utilizar generadores de números aleatorios criptográficamente seguros como `secrets.token_bytes()`, que proporciona valores impredecibles adecuados para cifrado. Si se necesita un keystream determinístico basado en una clave, es mejor utilizar funciones de derivación de claves como PBKDF2, scrypt, o HKDF, que fortalecen la clave original contra ataques de fuerza bruta. Otra opción segura es emplear un cifrador de bloque como AES en modo CTR, que genera un keystream al cifrar un contador incremental, asegurando que la salida sea resistente a ataques criptográficos.



**Referencias**
- Schneier, B. (1996). Applied Cryptography: Protocols, Algorithms, and Source Code in C. Wiley.
- Menezes, A. J., van Oorschot, P. C., & Vanstone, S. A. (1996). Handbook of Applied Cryptography